In [1]:
import os
from glob import glob
from tqdm import tqdm
import cv2
import pickle
import numpy as np

In [2]:

MODEL = 'OCR_dyunetCBAM_swinB.py' # model config 경로
PATHS = ['/opt/ml/segmentation/moon/mmsegmentation/work_dirs/OCR_dyunetCBAM_swinB_cv1',
        '/opt/ml/segmentation/moon/mmsegmentation/work_dirs/OCR_dyunetCBAM_swinB_cv2',
        '/opt/ml/segmentation/moon/mmsegmentation/work_dirs/OCR_dyunetCBAM_swinB_cv3',
        '/opt/ml/segmentation/moon/mmsegmentation/work_dirs/OCR_dyunetCBAM_swinB_cv4',
        '/opt/ml/segmentation/moon/mmsegmentation/work_dirs/OCR_dyunetCBAM_swinB_cv5',
        # '/opt/ml/segmentation/moon/mmsegmentation/work_dirs/pseudo_OCR_dyunetCBAM_swinB_smallMS_cv1'
] # 모델 저장된 폴더

weights = [0.743,0.756,0.754,0.758,0.748] # lb score
assert len(weights)==len(PATHS)

LOGIT_PATHS = [glob(os.path.join(path,'output*')) for path in PATHS]

logits=None
for idx,logit_path in enumerate(tqdm(LOGIT_PATHS)):
    assert len(logit_path)==1
    logit_path = logit_path[0]

    with open(logit_path,'rb') as f:
        logit = np.array(pickle.load(f))

    if idx==0:
        logits=logit
    else:
        logits = logits + logit
        
logits/=len(LOGIT_PATHS)

100%|██████████| 5/5 [03:16<00:00, 39.34s/it]


---
추가 output

In [3]:
path = glob(os.path.join('/opt/ml/segmentation/moon/mmsegmentation/work_dirs/pseudo_OCR_dyunetCBAM_swinB_cv1','output*'))
assert len(path)==1
logit_path = path[0]

with open(logit_path,'rb') as f:
    logit = np.array(pickle.load(f))

logits = logits+logit
logits/=2

---

In [4]:
output = [[] for _ in range(len(logits))]
for idx in tqdm(range(len(logits))):
    output[idx] = logits[idx].squeeze().argmax(axis=0)

100%|██████████| 819/819 [00:26<00:00, 31.10it/s]


In [9]:
pseudo_path = '/opt/ml/segmentation/moon/dataset/annotations/pseudo/'
DATAPATH = '/opt/ml/segmentation/moon/dataset/images/test'
file_names = sorted([ os.path.splitext(path)[0] for path in os.listdir(DATAPATH)])

for file_name,img in zip(file_names,output):
    file_path = os.path.join(pseudo_path,file_name+'.png')
    cv2.imwrite(file_path,img)

---

In [5]:
import albumentations as A
import pandas as pd

In [6]:
size = 256
transform = A.Compose([A.Resize(size, size)])

DATAPATH = '/opt/ml/segmentation/moon/dataset/images/test'
file_names = sorted([ os.path.splitext(path)[0] for path in os.listdir(DATAPATH)])

preds_array = np.empty((0,size*size),dtype=np.int64)

for idx,out in tqdm(enumerate(output)):
    image = np.zeros((1,size,size))
    transformed = transform(image=image,mask=out)
    mask = transformed['mask']
    
    mask = mask.reshape(-1,size*size).astype(int)
    preds_array = np.vstack((preds_array,mask))

819it [05:57,  2.29it/s]


In [7]:
NAME = 'OCR_dyunetCBAM_swinB_cv1-5+pseudo_cv1'

In [8]:
# sample_submisson.csv 열기
tmp_submission = pd.read_csv('/opt/ml/segmentation/baseline_code/submission/sample_submission.csv', index_col=None)

# PredictionString 대입
for file_name, string in zip(file_names, preds_array):
    file_name = '/'.join(file_name.split('+')) +'.jpg'
    tmp_submission = tmp_submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장

# 순서도 같아야 채점이 되기 때문에 sample_submission이 필요
sample = pd.read_csv('/opt/ml/segmentation/moon/submission/sample_submission.csv', index_col=None)
submission = pd.read_csv('/opt/ml/segmentation/baseline_code/submission/sample_submission.csv', index_col=None)
for image_id in sample['image_id'].tolist():
    prediction_string = tmp_submission[tmp_submission['image_id']==image_id]['PredictionString'].iloc[0]
    submission = submission.append({"image_id" : image_id, "PredictionString" : prediction_string }, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(f"./submission/ensemble_{NAME}.csv", index=False)